In [1]:
import sqlite3
import pandas as pd

In [2]:
# Connect to the SQLite database
connection = sqlite3.connect("dataset/FPA_FOD_20170508.sqlite")

query = "SELECT name FROM sqlite_master WHERE type='table';"
tables = connection.execute(query).fetchall()
tables_to_keep = {"NWCG_UnitIDActive_20170109", "Fires"}

for table in tables:
    table_name = table[0]
    if table_name not in tables_to_keep:
        try:
            drop_query = f"DROP TABLE {table_name};"
            connection.execute(drop_query)
            connection.commit()
        except Exception as e:
            print(f"Could not drop table {table_name}: {e}")

# Verify remaining tables
remaining_tables = connection.execute(query).fetchall()

print("\nRemaining tables in the database:")
for table in remaining_tables:
    print(table[0])

Could not drop table sqlite_sequence: table sqlite_sequence may not be dropped
Could not drop table SpatialIndex: no such module: VirtualSpatialIndex
Could not drop table ElementaryGeometries: no such module: VirtualElementary
Could not drop table KNN: no such module: VirtualKNN

Remaining tables in the database:
sqlite_sequence
SpatialIndex
ElementaryGeometries
KNN
Fires
NWCG_UnitIDActive_20170109


In [48]:
df_1 = pd.read_sql("SELECT * FROM Fires", con=connection); 
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1880465 entries, 0 to 1880464
Data columns (total 39 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   OBJECTID                    int64  
 1   FOD_ID                      int64  
 2   FPA_ID                      object 
 3   SOURCE_SYSTEM_TYPE          object 
 4   SOURCE_SYSTEM               object 
 5   NWCG_REPORTING_AGENCY       object 
 6   NWCG_REPORTING_UNIT_ID      object 
 7   NWCG_REPORTING_UNIT_NAME    object 
 8   SOURCE_REPORTING_UNIT       object 
 9   SOURCE_REPORTING_UNIT_NAME  object 
 10  LOCAL_FIRE_REPORT_ID        object 
 11  LOCAL_INCIDENT_ID           object 
 12  FIRE_CODE                   object 
 13  FIRE_NAME                   object 
 14  ICS_209_INCIDENT_NUMBER     object 
 15  ICS_209_NAME                object 
 16  MTBS_ID                     object 
 17  MTBS_FIRE_NAME              object 
 18  COMPLEX_NAME                object 
 19  FIRE_YEAR            

In [3]:
df_2 = pd.read_sql("SELECT * FROM NWCG_UnitIDActive_20170109", con=connection); 
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5867 entries, 0 to 5866
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   OBJECTID        5867 non-null   int64 
 1   UnitId          5867 non-null   object
 2   GeographicArea  5867 non-null   object
 3   Gacc            5867 non-null   object
 4   WildlandRole    5867 non-null   object
 5   UnitType        5867 non-null   object
 6   Department      5517 non-null   object
 7   Agency          5469 non-null   object
 8   Parent          0 non-null      object
 9   Country         5867 non-null   object
 10  State           5867 non-null   object
 11  Code            5867 non-null   object
 12  Name            5867 non-null   object
dtypes: int64(1), object(12)
memory usage: 596.0+ KB
